# Causal Inference on Churn Dataset

## Background

## Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
from dowhy import CausalModel
from sklearn.preprocessing import LabelEncoder

In [2]:
filepath = "Telco_customer_churn.xlsx"
df = pd.read_excel(filepath)

pd.set_option('display.max_columns', 33)
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [3]:
# lets look at aggregate stats of the numeric values
df[['Tenure Months', 'Monthly Charges', "Churn Value", "Churn Score", "CLTV"]].describe()

,Tenure Months,Monthly Charges,Churn Value,Churn Score,CLTV
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,32.371149,64.761692,0.265370,58.699418,4400.295755
std,24.559481,30.090047,0.441561,21.525131,1183.057152
min,0.000000,18.250000,0.000000,5.000000,2003.000000
25%,9.000000,35.500000,0.000000,40.000000,3469.000000
50%,29.000000,70.350000,0.000000,61.000000,4527.000000
75%,55.000000,89.850000,1.000000,75.000000,5380.500000
max,72.000000,118.750000,1.000000,100.000000,6500.000000


In [4]:
# check for null values
df.isnull().sum()

CustomerID              0
Count                   0
Country                 0
State                   0
City                    0
Zip Code                0
Lat Long                0
Latitude                0
Longitude               0
Gender                  0
Senior Citizen          0
Partner                 0
Dependents              0
Tenure Months           0
Phone Service           0
Multiple Lines          0
Internet Service        0
Online Security         0
Online Backup           0
Device Protection       0
Tech Support            0
Streaming TV            0
Streaming Movies        0
Contract                0
Paperless Billing       0
Payment Method          0
Monthly Charges         0
Total Charges           0
Churn Label             0
Churn Value             0
Churn Score             0
CLTV                    0
Churn Reason         5174
dtype: int64

In [5]:
# Lets replace null values in 'Churn Reason' with 'Don't Know', since it is an already existing value
print("Number of 'Don't know' responses: ", df["Churn Reason"].value_counts()["Don't know"])
print("Number of Null responses: ", df["Churn Reason"].isnull().sum())

df["Churn Reason"] = df["Churn Reason"].fillna("Don't know")

print("New Number of 'Don't know' responses: ", df["Churn Reason"].value_counts()["Don't know"])
print("New Number of Null responses: ", df["Churn Reason"].isnull().sum())

Number of 'Don't know' responses:  154
Number of Null responses:  5174
New Number of 'Don't know' responses:  5328
New Number of Null responses:  0


## Selecting Relevant Features

We need to simplify our model since there are a lot of features. We also need to create a basic DAG (Directed Acyclic Graph) to visualize the features that contribute to the outcome. The DAG will also allow us to identify potential treatment features and confounders to control.

In [249]:
# our columns
df.columns

Index(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code',
       'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen',
       'Partner', 'Dependents', 'Tenure Months', 'Phone Service',
       'Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
       'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method',
       'Monthly Charges', 'Total Charges', 'Churn Label', 'Churn Value',
       'Churn Score', 'CLTV', 'Churn Reason'],
      dtype='object')

Here is an ERD I created for better understanding of each of the columns

![ERD](images/erd.png)

In [6]:
# lets drop columns that are clearly not nessesary, the data only contains Californian customers
df.drop(["CustomerID", "Count", "Country", "State", "Lat Long"], axis=1, inplace=True)

While we can't really make use of the locational data, we can add external sources of data to create a new field that will tell us whether the customer resides in a rural or urban area of California by using publically avaiable population data of each Californian ZIP code. 

The goal here is to classify each city/town into their own population range, using the following arbitrarily created ranges:

- Rural: less than 1000 residents
- Town: 1000 to 25000 residents
- Small City: 25001 to 100000 residents
- Medium City: 100001 to 500000 residents
- Large City: Over 500000 residents

With telecommunications, it is important to understand what type of location the users are from, since more rural users require different usage plans than city users. 

In [251]:
population_filepath = 'zip-codes-data.csv'
pop = pd.read_csv(population_filepath)
pop.head()

,zip,population,city,county,state
0,90011,106042,Los Angeles,Los Angeles County,California
1,90650,101983,Norwalk,Los Angeles County,California
2,94565,100826,Pittsburg,Contra Costa County,California
3,92336,100571,Fontana,San Bernardino County,California
4,91331,99804,Pacoima,Los Angeles County,California


In [252]:
# we only need the city and population columns
pop = pop[["city", "population"]]

# rename the columns for joining and consistency
pop = pop.rename(columns={
    "city": "City",
    "population": "Population"
})

pop = pop.groupby('City', as_index=False)['Population'].sum()

In [253]:
# some cities have been integrated or grouped into other cities, while some just need to change the inconsistent formatting
# this is to ensure that when they are joined, the population do not become NaN
rename_cities = {
    'Sun City': 'Menifee',
    'White Water': 'Whitewater',
    'Bell': 'Los Angeles',
    'Mc Farland': 'McFarland'
}

df['City'] = df['City'].replace(rename_cities)

In [254]:
# join the cities with population data
new_df = df.join(pop.set_index('City'), on='City', how='left')

There are some areas in the city column that are NaN. I did not include some of these cities in the `rename_cities` dictionary because they are extremely small communities with fewer than 100 people. They can simply be designated as rural.

In [255]:
# Because some cities are not correctly designated in the 
null_cities = new_df.loc[new_df['Population'].isnull()]
null_cities["City"].unique()

array(['Ludlow', 'Amboy', 'Mc Kittrick', 'Sheep Ranch', 'Piercy',
       'Echo Lake', 'Campo Seco', 'Pinecrest', 'Parker Dam', 'Vidal',
       'Mill Creek', 'O Neals', 'Duncans Mills', 'Phillipsville',
       'Smartville'], dtype=object)

In [256]:
# lets map the populations into their specific location
urban_cat = {
    'Rural': (0, 999),
    'Town': (1000, 20000),
    'Small City': (20001, 100000),
    'Medium City': (100001, 300000),
    'Large City': (300001, float('inf'))
}

def categorize_population(population):
    if population == np.nan:
        return 'Rural'
    for category, (lower, upper) in urban_cat.items():
        if lower <= population <= upper:
            return category

new_df['Urban Category'] = new_df['Population'].apply(categorize_population)

In [257]:
new_df['Urban Category'].value_counts()

Urban Category
Town           1896
Small City     1711
Large City     1156
Medium City    1116
Rural          1101
Name: count, dtype: int64

In [258]:

df = df[['Churn Value', 'Internet Service', 'Senior Citizen', 'Monthly Charge', 'Tenure Months', 'Contract']]

# Handle categorical variables (e.g., Internet Service and Contract)
df['Internet Service'] = LabelEncoder().fit_transform(df['Internet Service'])
df['Contract'] = LabelEncoder().fit_transform(df['Contract'])
df['Senior Citizen'] = df['Senior Citizen'].apply(lambda x: 1 if x == 'Yes' else 0)

# Treatment: Internet Service (e.g., whether the customer uses Fiber Optic)
# Outcome: Churn Value (whether the customer churned or not)
# Confounders: Senior Citizen, Monthly Charge, Tenure Months, Contract Type

# Define the causal model
model = CausalModel(
    data=df,
    treatment='Internet Service',
    outcome='Churn Value',
    common_causes=['Senior Citizen', 'Monthly Charge', 'Tenure Months', 'Contract']
)

# View the model
model.view_model()

# Identify the causal effect using the back-door criterion
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

# Estimate the causal effect
causal_estimate = model.estimate_effect(identified_estimand,
                                        method_name="backdoor.propensity_score_matching")
print(causal_estimate)

# Refute the estimate (to check robustness)
refutation = model.refute_estimate(identified_estimand, causal_estimate, method_name="placebo_treatment_refuter")
print(refutation)


KeyError: "['Monthly Charge'] not in index"

In [209]:
df.columns

Index(['City', 'Zip Code', 'Lat Long', 'Latitude', 'Longitude', 'Gender',
       'Senior Citizen', 'Partner', 'Dependents', 'Tenure Months',
       'Phone Service', 'Multiple Lines', 'Internet Service',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing',
       'Payment Method', 'Monthly Charges', 'Total Charges', 'Churn Label',
       'Churn Value', 'Churn Score', 'CLTV', 'Churn Reason'],
      dtype='object')